### ReS2 Slip Vector

In [48]:
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.models as models
from model import StructNet
import numpy as np
from math import cos, sin, tan,degrees, radians

def shift_trans(x,y):
    all_targets = np.array([[x,y,1.]])

    all_targets_symmetric = all_targets.copy()
    all_targets_symmetric[:, 1] = -all_targets_symmetric[:, 1]

    all_targets_filp = all_targets_symmetric.copy()
    all_targets_filp[:, :2] = (0, 0) - (all_targets_symmetric[:, :2] - (0, 0))
    points = np.concatenate([all_targets_filp,all_targets_symmetric])

    condition_indices = np.where(points[:, 0]<0)

    shift_value = 6.42
    points[condition_indices, 0] += shift_value 
    condition_indices = np.where(points[:, 1]<0)
    points[condition_indices, 0] += -3.21
    points[condition_indices, 1] += 5.71
    condition_indices = np.where(1.74*points[:,0]+points[:, 1]<0)
    points[condition_indices, 0] += 6.42
    condition_indices2 = np.where(1.74*points[:,0]+points[:, 1]>11.21)
    points[condition_indices2, 0] += -6.42

    index = []
    for i in range(points.shape[0]):
        if (5.71/(3.27+6.42))*points[i][0] + points[i][1] > (5.71/(3.27+6.42))*6.42:
            index.append(i)

    points = np.delete(points,index,axis=0)

    angle_in_degrees1 = 119.8- 90
    angle_in_radians1 = radians(angle_in_degrees1)
    angle_in_degrees2 = 60.2
    angle_in_radians2 = radians(angle_in_degrees2)
    new_x = [p[0]+p[1]*tan(angle_in_radians1) for p in points]
    new_y = [p[1]/cos(angle_in_radians1) for p in points]
    target_new = np.stack([new_x, new_y],axis=1)

    return target_new[0]

weight_path = '/vepfs/fs_ckps/ycjin/nudt/sim/ablation_update/model_params-800.pth'
image_path = '/vepfs/fs_users/ycjin/3.png'


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StructNet(output_dim = 3)
model.to(device)

model.load_state_dict(torch.load(weight_path))
model.eval()

image = Image.open(image_path).convert('RGB')
image = image.resize((1024,1024))
# image = TF.hflip(image)
image_list = []

for i in range(16):
    crop_size = 512 + i * 10
    image_c = TF.crop(image, 0, 0, crop_size, crop_size)
    image_c = image_c.resize([512,512])
    data = TF.to_tensor(image_c)
    image_list.append(data)
data = torch.stack(image_list,dim = 0).to(device)
output = model(data)
output = output.mean(dim=0)[:2].cpu().detach()
shift_trans(output[0], output[1])

array([1.16022873, 0.07804659])

In [13]:
shift_trans(-1.2693,  0.0666)

array([1.30744218, 0.07674885])

In [49]:
model(data)[:,:2].cpu().detach()

tensor([[-2.7790,  0.1226],
        [-2.8051,  0.0977],
        [-2.7986,  0.0947],
        [-2.8250,  0.0714],
        [-1.6443,  0.0890],
        [ 0.8163,  0.0654],
        [-0.7207,  0.0639],
        [-0.8000,  0.0326],
        [ 0.8027,  0.0583],
        [-1.2693,  0.0666],
        [-2.2677,  0.0508],
        [-1.7738,  0.0287],
        [-2.7097,  0.0559],
        [-0.8225,  0.0583],
        [ 1.7957,  0.0450],
        [ 1.8579,  0.0826]])

In [51]:
#Kmeans
from sklearn.cluster import KMeans
from sklearn import preprocessing
k_model = KMeans(n_clusters=3)
k_model.fit(model(data).cpu().detach())
predict_y = k_model.predict(model(data).cpu().detach())
predict_y


array([0, 0, 0, 0, 2, 1, 2, 2, 1, 2, 0, 2, 0, 2, 1, 1], dtype=int32)